<a href="https://colab.research.google.com/github/MOOSUNGPARK/education/blob/master/segmentation_lung_cancer2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/deepnoid/education.git

In [0]:
from keras.models import Model
from keras.layers import *
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
import keras.backend as K
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob
from skimage.transform import rotate
from skimage import exposure

In [0]:
BASE_PATH = r'./education/7. Deep Learning Keras/dataset/segmentation'
IMG_DATA_PATH = os.path.join(BASE_PATH, 'lung_cancer', 'input', '2d_images')
MASK_DATA_PATH = os.path.join(BASE_PATH, 'lung_cancer', 'input', '2d_masks')

In [0]:
#@title
def left_shifting(img, shift_rate=0.1):
    h, w = img.shape
    margin = int(w*shift_rate)
    img_with_margin = np.pad(img, ((0, 0), (0, margin)), 'constant')
    img = img_with_margin[:,margin:] 
    return img 
  
def right_shifting(img, shift_rate=0.1):
    h, w = img.shape
    margin = int(w*shift_rate)
    img_with_margin = np.pad(img, ((0, 0), (margin, 0)), 'constant')
    img = img_with_margin[:,:w]
    return img 

def up_shifting(img, shift_rate=0.1):
    h, w = img.shape
    margin = int(h*shift_rate)
    img_with_margin = np.pad(img, ((0, margin), (0, 0)), 'constant')
    img = img_with_margin[margin:,:]
    return img 

def down_shifting(img, shift_rate=0.1):
    h, w = img.shape
    margin = int(h*shift_rate)
    img_with_margin = np.pad(img, ((margin, 0), (0, 0)), 'constant')
    img = img_with_margin[:h,:]
    return img 
  
def erasing(image_origin, p=0.5, s=(0.04, 0.08), r=(0.5, 1.5), mask_value='random'):
    image = np.copy(image_origin)
    if mask_value == 'mean':
        mask_value = image.mean()
    elif mask_value == 'random':
        mask_value = np.random.randint(0, 1)
    h, w = image.shape
    mask_area = np.random.randint(h * w * s[0], h * w * s[1])
    mask_aspect_ratio = np.random.rand() * r[1] + r[0]
    mask_height = int(np.sqrt(mask_area / mask_aspect_ratio))
    if mask_height > h - 1:
        mask_height = h - 1
    mask_width = int(mask_aspect_ratio * mask_height)
    if mask_width > w - 1:
        mask_width = w - 1

    top = np.random.randint(0, h - mask_height)
    left = np.random.randint(0, w - mask_width)
    bottom = top + mask_height
    right = left + mask_width
    image[top:bottom, left:right].fill(mask_value)
    return image
  
def rotating(img, angle=10):
    return rotate(img, angle)

def lr_flip(img):
    return np.fliplr(img)
  
def ud_flip(img):
    return np.flipud(img)

def load_img(path, height=96, width=96):
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    return cv2.resize(img, dsize=(height, width), interpolation=cv2.INTER_AREA)
  
def load_imgs(paths, height=96, width=96):
    return np.stack([load_img(path, height, width) for path in paths])[:,:,:,np.newaxis]

def load_mask(path, height=96, width=96):
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED).astype(np.float32) / 255.
    return cv2.resize(img, dsize=(height, width), interpolation=cv2.INTER_AREA)

def load_masks(paths, height=96, width=96):
    return np.stack([load_mask(path, height, width) for path in paths])[:,:,:,np.newaxis]

def show_img(img):
    plt.imshow(img, cmap='Greys')
    plt.show()
    
def show_mask(mask):
    plt.imshow(mask, cmap='gray')
    plt.show()
    
def show_imgs(imgs):
    for idx in range(5):
        plt.imshow(imgs[idx,:,:,0], cmap='Greys')
        plt.show()

def show_masks(mask):
    for idx in range(5):
        plt.imshow(masks[idx,:,:,0], cmap='gray')
        plt.show()
        
def conv_layer(layer, filters, activation, kernel_size=(3, 3)):
    layer = Conv2D(filters=filters, kernel_size=kernel_size, activation=None, padding='same')(layer)
    layer = BatchNormalization()(layer)
    layer = Activation(activation)(layer)
    return layer

def make_unet(imgs, filters=32, activation='elu'):
    
    input_layer = Input(shape=imgs.shape[1:])

    c1 = conv_layer(layer=input_layer, filters=filters, activation=activation)
    l = MaxPool2D(strides=(2, 2))(c1)

    filters *= 2
    c2 = conv_layer(layer=l, filters=filters, activation=activation)
    l = MaxPool2D(strides=(2, 2))(c2)

    filters *= 2
    c3 = conv_layer(layer=l, filters=filters, activation=activation)
    l = MaxPool2D(strides=(2, 2))(c3)

    filters *= 2
    c4 = conv_layer(layer=l, filters=filters, activation=activation)
    l = concatenate([UpSampling2D(size=(2, 2))(c4), c3], axis=-1)

    l = conv_layer(layer=l, filters=filters, activation=activation)
    l = concatenate([UpSampling2D(size=(2, 2))(l), c2], axis=-1)

    filters //= 2
    l = conv_layer(layer=l, filters=filters, activation=activation)
    l = concatenate([UpSampling2D(size=(2, 2))(l), c1], axis=-1)

    filters //= 2
    l = conv_layer(layer=l, filters=filters, activation=activation)

    filters //= 2
    l = conv_layer(layer=l, filters=filters, activation=activation)

    output_layer = Conv2D(filters=1, kernel_size=(1, 1), activation='sigmoid')(l)

    model = Model(input_layer, output_layer)
    model.summary()
    return model
  
def train_unet(imgs, masks, model, batch_size=5, epochs=10, learning_rate=1e-4, decay_rate=0.9, val_ratio=0.2):
    imgs_train, imgs_val, masks_train, masks_val = train_test_split(imgs, masks, test_size=val_ratio)
    def dice_score(y_true, y_pred):
        y_true_f = K.flatten(y_true)
        y_pred_f = K.flatten(y_pred)
        intersection = K.sum(y_true_f * y_pred_f)
        return (2. * intersection + K.epsilon()) / (K.sum(y_true_f) + K.sum(y_pred_f) + K.epsilon())
  
    model.compile(optimizer=Adam(learning_rate), loss='binary_crossentropy', metrics=[dice_score])
    weight_saver = ModelCheckpoint('lung.h5', monitor='val_dice_score', save_best_only=True, save_weights_only=True)
    decayed_lr = LearningRateScheduler(lambda x: learning_rate * decay_rate ** x)
    
    history = model.fit(imgs_train, masks_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(imgs_val, masks_val),
                    shuffle=True,
                    verbose=2,
                    callbacks=[weight_saver, decayed_lr])
    return history
  
def show_results(history):
    dice_score = history.history['dice_score']
    val_dice_score = history.history['val_dice_score']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(dice_score))
    
    plt.plot(epochs, dice_score, 'b', color='blue', label='Training score')
    plt.plot(epochs, val_dice_score, 'b', color='red', label='Validation score')
    plt.title('Training and validation score')
    plt.legend()
    plt.show()
    
    
    plt.plot(epochs, loss, 'b', color='blue', label='Training loss')
    plt.plot(epochs, val_loss, 'b', color='red', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    plt.show()
    
def show_result_imgs(imgs, masks):
    fix, ax = plt.subplots(5, 3, figsize=(10,20))
    for i in range(5):
        pred = model.predict(imgs[i][np.newaxis, :, : ,:])
        mask = (pred >= 0.5).astype(np.uint8)
        ax[i, 0].imshow(imgs[i, :, :, 0], cmap='gray')
        ax[i, 1].imshow(masks[i, :, :, 0], cmap='gray')
        ax[i, 2].imshow(mask[0, :, :, 0], cmap='gray')
    plt.show()

In [0]:
img_paths = sorted(glob.glob(os.path.join(IMG_DATA_PATH, '*.tif')))
mask_paths = sorted(glob.glob(os.path.join(MASK_DATA_PATH, '*.tif')))
img_path = img_paths[0]
mask_path = mask_paths[0]

#이미지 불러오기

In [0]:
imgs = load_imgs(img_paths)
masks = load_masks(mask_paths)
img = load_img(img_path)
mask = load_mask(mask_path)

#이미지 전처리하기

###1. 이동
img 위 : img=up_shifting(img)  
img 아래 : img=down_shifting(img)  
img 왼쪽 : img=left_shifting(img)  
img 오른쪽 : img=right_shifting(img)   

mask 위 : img=up_shifting(mask)  
mask 아래 : img=down_shifting(mask)  
mask 왼쪽 : img=left_shifting(mask)  
mask 오른쪽 : img=right_shifting(mask)  

###2. 회전
img 10도 회전 : img=rotating(img, angle=10)    
img 90도 회전 : img=rotating(img, angle=90)  

mask 10도 회전 : mask=rotating(mask, angle=10)  
mask 90도 회전: mask=rotating(mask, angle=90)  

###3. 반전
img 상하 : img=ud_filp(img)  
img 좌우: img=lr_flip(img)  

mask 상하 : mask=ud_flip(mask)  
mask 좌우: mask=lr_flip(mask)

###4. 일부 지우기
img 지우기 : img=erasing(img)  

mask 지우기 : mask=erasing(mask)

###5. 이미지 출력
img 출력 : show_img(img)  

mask 출력 : show_mask(mask)




  

In [0]:
img=up_shifting(img)
show_img(img)

#이미지 확인하기

In [0]:
show_imgs(imgs)

#U-net 만들기

In [0]:
model = make_unet(imgs)

#U-net 학습하기

In [0]:
history = train_unet(imgs, masks, model)

# 결과 확인하기

In [0]:
show_results(history)

#결과 이미지 출력

In [0]:
show_result_imgs(imgs, masks)